# Imports and Data

In [1]:
import pandas as pd
import requests
import time
import json
import numpy as np


from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By


from selenium.common.exceptions import NoSuchElementException,TimeoutException

options = Options()
options.add_argument("start-maximized")
options.add_experimental_option("detach", True)
ser = Service("C:\Program Files (x86)\chromedriver.exe")


In [2]:
source_data = pd.read_excel("../../Data/ERA_Alabama.xlsx",sheet_name="All Plants")


In [86]:
temp = pd.read_csv("SearchResults.csv")
symbols_in_USDA = temp["Accepted Symbol"]

In [3]:
symbols = source_data["USDA Symbol"]

In [87]:
symbols_in_both = symbols[symbols.isin(symbols_in_USDA)]

# Navigate to page

In [47]:
driver = webdriver.Chrome(service=ser, options=options)
driver.get(f"https://plants.usda.gov/home/plantProfile?symbol={symbols[2]}")

# Scrape Plant

In [36]:
charactersitics_div = driver.find_element(By.ID,"characteristics")
source = charactersitics_div.get_attribute("innerHTML")
table = pd.read_html(source)[0]
table.columns = ["Column","Value"]

In [37]:
group_index = (table["Column"]==table["Value"])


In [38]:
group_index

0      True
1     False
2     False
3     False
4     False
      ...  
79    False
80    False
81    False
82    False
83    False
Length: 84, dtype: bool

In [39]:
table = table[~group_index]

In [40]:
table.set_index("Column",inplace=True)

In [41]:
table = table.T

In [42]:
table

Column,Active Growth Period,After Harvest Regrowth Rate,Bloat,C:N Ratio,Coppice Potential,Fall Conspicuous,Fire Resistant,Flower Color,Flower Conspicuous,Foliage Color,...,Lumber Product,Naval Store Product,Nursery Stock Product,Palatable Browse Animal,Palatable Graze Animal,Palatable Human,Post Product,Protein Potential,Pulpwood Product,Veneer Product
Value,"Spring, Summer, Fall",Slow,None,High,No,Yes,No,Red,Yes,Green,...,No,No,Yes,Low,Low,No,No,Low,No,No


In [44]:
table.index = [symbols[2]]
table.columns.names = [None]

In [45]:
table

,Active Growth Period,After Harvest Regrowth Rate,Bloat,C:N Ratio,Coppice Potential,Fall Conspicuous,Fire Resistant,Flower Color,Flower Conspicuous,Foliage Color,...,Lumber Product,Naval Store Product,Nursery Stock Product,Palatable Browse Animal,Palatable Graze Animal,Palatable Human,Post Product,Protein Potential,Pulpwood Product,Veneer Product
ACVI,"Spring, Summer, Fall",Slow,None,High,No,Yes,No,Red,Yes,Green,...,No,No,Yes,Low,Low,No,No,Low,No,No


# Scrape Sample of Plants

In [53]:
list_of_dfs = []

In [ ]:
symbol_sample = symbols[:8]

driver = webdriver.Chrome(service=ser, options=options)

wait = WebDriverWait(driver, 3)

for symbol in symbol_sample:

    print(symbol)

    driver.get(f"https://plants.usda.gov/home/plantProfile?symbol={symbol}")

    time.sleep(5)

    try:
        characteristics_div = wait.until(EC.presence_of_element_located(((By.ID,"characteristics"))))   

    except TimeoutException:
        continue

    # charactersitics_div = driver.find_element(By.ID,"characteristics")


    source = charactersitics_div.get_attribute("innerHTML")
    table = pd.read_html(source)[0]
    table.columns = ["Column","Value"]

    group_index = (table["Column"]==table["Value"])
    table = table[~group_index]

    table.set_index("Column",inplace=True)
    table = table.T
    table.index = [symbol]
    table.columns.names = [None]

    list_of_dfs.append(table)


full_df = pd.concat(list_of_dfs)
full_df.iloc[:,:10]


    

In [71]:
full_df.to_csv("TEST.csv")

# Scrape All

In [89]:
list_of_dfs = []

In [96]:
len(list_of_dfs)

199

In [94]:
symbols_in_both[189:]

443     CYPS
446     CYST
450     CYRA
457     DACO
462     DEVE
        ... 
1607    VIVU
1610    WIFR
1617    YUAL
1624    ZIAQ
1625    ZIMI
Name: USDA Symbol, Length: 441, dtype: object

Stopped at CYPS
DILA8


In [115]:
symbols_in_both.to_list().index("DIOV")

203

In [138]:
errors = []

In [139]:
driver = webdriver.Chrome(service=ser, options=options)
wait = WebDriverWait(driver, 3)

for symbol in symbols_in_both[206:]:

    print(symbol)

    time.sleep(5)

    
    driver.get(f"https://plants.usda.gov/home/plantProfile?symbol={symbol}")

    try:
        wait.until(EC.presence_of_element_located(((By.ID,"characteristics"))))  

    except TimeoutException:
        errors.append(symbol)
        continue



    charactersitics_div = driver.find_element(By.ID,"characteristics")
   
    source = charactersitics_div.get_attribute("innerHTML")

    
    try:
        table = pd.read_html(source)[0]
    
    except:
        errors.append(symbol)
        continue
  

    table.columns = ["Column","Value"]

    group_index = (table["Column"]==table["Value"])
    table = table[~group_index]

    table.set_index("Column",inplace=True)
    table = table.T
    table.index = [symbol]
    table.columns.names = [None]

    list_of_dfs.append(table)

    print(errors)


full_df = pd.concat(list_of_dfs)
full_df.iloc[:,:10]

DILO
[]
DICI
[]
DIVI5
[]
DISP
DIFR6
['DISP']
DOME
['DISP']
DRAM
['DISP']
DRCR4
DUAR3
['DISP', 'DRCR4']
ECPA
['DISP', 'DRCR4']
ECPU
ECWA
ECPR
['DISP', 'DRCR4', 'ECPU', 'ECWA']
ELAC
ELCE
ELCO2
ELEQ
['DISP', 'DRCR4', 'ECPU', 'ECWA', 'ELAC', 'ELCE', 'ELCO2']
ELOB2
['DISP', 'DRCR4', 'ECPU', 'ECWA', 'ELAC', 'ELCE', 'ELCO2']
ELPA3
['DISP', 'DRCR4', 'ECPU', 'ECWA', 'ELAC', 'ELCE', 'ELCO2']
ELPA5
ELQU
['DISP', 'DRCR4', 'ECPU', 'ECWA', 'ELAC', 'ELCE', 'ELCO2', 'ELPA5']
ELRO2
['DISP', 'DRCR4', 'ECPU', 'ECWA', 'ELAC', 'ELCE', 'ELCO2', 'ELPA5']
ELTE
ELCA3
['DISP', 'DRCR4', 'ECPU', 'ECWA', 'ELAC', 'ELCE', 'ELCO2', 'ELPA5', 'ELTE']
ELCA7
['DISP', 'DRCR4', 'ECPU', 'ECWA', 'ELAC', 'ELCE', 'ELCO2', 'ELPA5', 'ELTE']
ELVI3
['DISP', 'DRCR4', 'ECPU', 'ECWA', 'ELAC', 'ELCE', 'ELCO2', 'ELPA5', 'ELTE']
EPCO
['DISP', 'DRCR4', 'ECPU', 'ECWA', 'ELAC', 'ELCE', 'ELCO2', 'ELPA5', 'ELTE']
EQAR
['DISP', 'DRCR4', 'ECPU', 'ECWA', 'ELAC', 'ELCE', 'ELCO2', 'ELPA5', 'ELTE']
ERHY
ERIN
['DISP', 'DRCR4', 'ECPU', 'ECWA', 'ELAC

,Active Growth Period,After Harvest Regrowth Rate,Bloat,C:N Ratio,Coppice Potential,Fall Conspicuous,Fire Resistant,Flower Color,Flower Conspicuous,Foliage Color
ACVI,"Spring, Summer, Fall",Slow,None,High,No,Yes,No,Red,Yes,Green
ACFL,Spring and Summer,NaN,None,High,No,Yes,No,Yellow,No,Green
ACNI5,Spring and Summer,NaN,None,High,No,Yes,No,Yellow,No,Green
ACRU,Spring and Summer,NaN,None,High,Yes,Yes,No,Red,Yes,Green
ACSA2,Spring and Summer,NaN,None,High,Yes,Yes,No,Yellow,Yes,Green
...,...,...,...,...,...,...,...,...,...,...
VIRO3,"Spring, Summer, Fall",NaN,None,High,No,Yes,No,White,No,Green
VIVU,"Spring, Summer, Fall",NaN,None,High,No,Yes,No,White,No,Green
WIFR,Spring,NaN,None,NaN,No,No,No,Purple,Yes,Green
YUAL,Spring,NaN,None,High,No,No,No,White,Yes,Dark Green


In [141]:
lst = ['DISP', 'DRCR4', 'ECPU', 'ECWA', 'ELAC', 'ELCE', 'ELCO2', 'ELPA5', 'ELTE', 'ERHY', 'ERPH', 'ERST3', 'ERDE5', 'EUFI14', 'FERU2', 'FRAM2', 'GAAP2', 'GAOB', 'GATI', 'GABA', 'GLTR', 'HEMA2', 'HIAC', 'HOPU', 'HYCR3', 'HYGA', 'HYMU', 'HYHI2', 'ILOP', 'JUNI', 'JUBI', 'JUBR2', 'JUTE', 'JUVA2', 'JUVI', 'LECA8', 'LEHI2', 'LICA17', 'LYCA2', 'LYLI', 'MAFR', 'MAVI2', 'MOFI', 'MYMI2', 'NAGU', 'NYSY', 'OEBI', 'OEPI2', 'OSCL2', 'PAAU3', 'PAGL17', 'PAAM2', 'PAHE2', 'PALA10', 'PAVA', 'PECA', 'PIEL', 'POAU', 'POSY', 'POPU5', 'PODE3', 'PRVU', 'PRAN3', 'QUMA3', 'QUMO4', 'QUPA5', 'QUPH', 'QUPU80', 'QUSH', 'QUST', 'QUVE', 'QUVI', 'RALA2', 'RHAR3', 'RHVI2', 'ROPS', 'ROPA', 'RUAL', 'RUAR2', 'RUCU', 'RUFL', 'RUOD', 'SAPA', 'SALA2', 'SCTE5', 'SCTA2', 'SCAT2', 'SERE2', 'SIAN3', 'SMGL', 'SMRO', 'SMWA', 'SOFL2', 'SOPA2', 'SORU2', 'SPAM', 'SPSP', 'SPVI3', 'STGR4', 'SYCO4', 'TAAS', 'TADI2', 'TECA3', 'TIAM', 'TRVI', 'TRST2', 'TRDA3', 'TSCA', 'ULAM', 'ULRU', 'URPL2', 'VAST', 'VEEN', 'ZIAQ']

with open("Errors.txt","w") as f:
    for error in errors:
        f.write(error + "\n")
        


In [140]:
full_df.to_csv("USDAPlants_Unedited.csv")